In [6]:
#------------------------------importing library---------------------------------------
from dateutil.parser import parse
import numpy as np
import pandas as pd
import pickle
# from bpemb import BPEmb
import pytesseract
import cv2
import glob
%matplotlib notebook
import matplotlib.pyplot as plt
from PIL import Image
from bs4 import BeautifulSoup
import re
from timeit import default_timer as timer


In [8]:
text_path = "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/*.txt"
img_path = "C:/Users/Sanjeev/Downloads/software/0325updated.task2train(626p)-20200414T125907Z-001/0325updated.task2train(626p)/*.jpg"
tesseract_path = 'C:/python_lib/ocr/tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = 'C:/python_lib/ocr/tesseract.exe'
file_save_path='C:/Users/Sanjeev/Documents/Python Scripts/Dataset/label_data/'

In [13]:
text_file = glob.glob(text_path)
img_file = glob.glob(img_path)

In [11]:
def SaveData(data,path):
    with open(path,'wb') as file:
        pickle.dump(data,file)

def LoadData(path):
    with open(path, 'rb') as file:
        return pickle.load(file)

def display_image(img):
    res = isinstance(img, str)
    if res:
        print('reading image ../n' )
        img = cv2.imread(img)  
    if len(img.shape) > 2:
        plt.imshow(img, cmap='gray')
        plt.title("Image")
        plt.axis('off')
    else:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        plt.title("RGB Image")
        plt.axis('off')


In [14]:
def get_ocr_text(img):  
    res = isinstance(img, str)
    if res:
        print('reading image ../n' )
        img = cv2.imread(img)  
    text = pytesseract.image_to_string(img)
#     print(text)
    return text

# read_data ()-> out data [image_list,ocr_text,out_text,file_name]
def read_data(img_path_list, text_path_list, data_no=150):
    img_list,ocr_text_list,text_list,file_name_list=[],[],[],[]
    j=0
    for i in range(data_no):
        img_path= img_path_list[i]
        img_name = img_path.split("\\")[-1]
        img_name = img_name.split(".jpg")[0]
        img_data = cv2.imread(img_path_list[i])
        img_list.append(img_data)
        ocr_text = get_ocr_text(img_path)
        ocr_text_list.append(ocr_text)
        text_path = text_path_list[j]
        text_name=text_path.split("\\")[-1]
        text_name = text_name.split('.txt')[0]
        while img_name != text_name:
            j +=1
            print('fileName is not same: ',img_name, text_name)
            text_path = text_path_list[j]
            text_name=text_path.split("\\")[-1]
            text_name = text_name.split('.txt')[0]
        with open(text_path, 'r') as file:
            text_data =file.read()
        text_list.append(eval(text_data))
        file_name_list.append(img_name)
#         data_list.append([img_data,ocr_text,eval(text_data),img_name])    
    return img_list,ocr_text_list,text_list,file_name_list


In [16]:
#Creating the treaining sequence for lstm
def create_training_seq(input_data,output_data,batch=False):
    if not batch:
        input_data=[input_data]
        output_data=[output_data]
    X_list =[]  # for storing the list of input_data sequence
    y_list= []
    for i in range(len(input_data)):
#         print("\n-----------------------------loop ",i,"---------------------------------")
        in_data= input_data[i]
        out_data= output_data[i]
        in_list,out_list,temp_list,value_idx,idx =[],[],[],0,1
        while idx < len(in_data):
    #     for idx in range(1, len(in_data)):    # it will not increase index:  the error 
            temp_list =temp_list+[in_data[idx-1]]
            in_list.append(temp_list)
            text= in_data[idx]
#            uncomment below line to see output 
#             print('\n-->in_list : ',in_list[idx-1])
#             print('\n-->out_list : ',out_list)
#             print('idx : ',idx)
            if text == out_data[value_idx][1][0]:
                out_list.append(out_data[value_idx][0])
                j=0
#                 while in_data[idx] != out_data[value_idx][1][-1]:
                while j < len(out_data[value_idx][1]) and in_data[idx] != out_data[value_idx][1][-1]:
                    temp_list =temp_list+[in_data[idx]]
                    in_list.append(temp_list)
                    out_list.append(in_data[idx])  # todo what should be the output data actural data or extracted one (now). 
                    idx +=1
                    j +=1
#                     print('idx : ',idx)
                temp_list =temp_list+[in_data[idx]]
                in_list.append(temp_list)
                out_list.append(in_data[idx])
                value_idx +=1
            else:
                out_list.append('<nothing>')
    #         print('in_list : ',in_list[idx-1])
    #         print('out_list : ',out_list)
            idx +=1
#         print('idx : ',idx)
        X_list.append(in_list)
        y_list.append(out_list)
        
    if not batch:
        return X_list[0],y_list[0] 
        
    return X_list,y_list
            

In [17]:
len(text_file),len(img_file)

(876, 735)